In [1]:
%%capture
%%bash
pip install -U vaex-core vaex-hdf5 dask numpy fsspec>=0.3.3 fastparquet pyarrow tqdm numpy
aws s3 cp s3://xdss-public-datasets/demos/taxi_1B.hdf5 datasets/taxi_1B.hdf5

In [24]:
import sys
import pandas as pd
import pyarrow
import pyarrow.parquet
from tqdm import tqdm
import gc
import vaex

data_path = '../datasets/taxi_1m.csv'
sample = pd.read_csv(data_path, nrows=100)
sample['pickup_hour'] = pd.to_datetime(sample.pickup_datetime).dt.hour
dtypes = {col: sample[col].dtype for col in sample.columns}


In [ ]:
SPLIT_ROWS = 1_000_000
!mkdir -p datasets/taxi_parquet

for i,chunk in enumerate(tqdm(pd.read_csv(data_path, chunksize=SPLIT_ROWS, dtype=dtypes))):
    chunk.to_parquet(f"datasets/taxi_parquet/data_{i}.parquet", engine='pyarrow')
    del chunk
    gc.collect()

In [34]:
import dask.dataframe as dd
df = dd.read_parquet('../datasets/taxi_parquet2/', engine='pyarrow')
len(df)

150000

In [ ]:
%%capture
!aws s3 cp --recursive ../datasets/taxi_parquet s3://xdss-public-datasets/demos/taxi_parquet